<a href="https://colab.research.google.com/github/yilewang/TVB_Demo/blob/master/sim2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
%pip install tvb-library
%pylab inline
import os
from tvb.simulator.lab import *
LOG = get_logger('demo')
import pickle as cPickle
from tvb.simulator.models.stefanescu_jirsa import ReducedSetHindmarshRose
import matplotlib.pyplot as plt
import numpy as np


In [2]:
%%capture
path = "C:/Users/wayne/tvb/tvbdemos/tvb_slurm/0306A.zip"

In [3]:
speed=10.
oscillator = models.ReducedSetHindmarshRose()
white_matter = connectivity.Connectivity.from_file(path)
# oscilator.variables_of_interest = ["xi"]
white_matter.speed = np.array([speed])
white_matter_coupling = coupling.Linear(a=np.array([0.04]))
heunint = integrators.HeunStochastic(dt=0.01220703125, noise=noise.Additive(nsig=np.array([0.00001]), ntau=0.0, random_stream=np.random.RandomState(seed=42)))
# Initialise some Monitors with period in physical time
mon_raw = monitors.Raw()
mon_tavg = monitors.TemporalAverage(period=1.)
what_to_watch = (mon_tavg, mon_raw)
# Initialise a Simulator -- Model, Connectivity, Integrator, and Monitors.
sim = simulator.Simulator(model=oscillator, connectivity=white_matter,
                        coupling=white_matter_coupling,
                        integrator=heunint, monitors=what_to_watch, simulation_length=1e3)
sim.configure()

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (<ipython-input-3-617ed1920188>, line 6)

In [ ]:
(tavg_time, tavg_data), (raw_time, raw_data) = sim.run()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(raw_time * 1e-3, raw_data[:, 0, :, 0], alpha=0.3)
plt.title('Initial transient in RAW')
plt.xlabel('Time (s)')
plt.legend()
plt.grid(True)
plt.show()